<a href="https://colab.research.google.com/github/saurograndi/healthcare_nlp/blob/main/Medical_Chatbot_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade pip
!pip install -q --upgrade langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.6 MB/s eta 0:00:00


Let's download 1000 abstract articles from PubMed diabetes related already cllected as csv

In [2]:
! wget -q https://raw.githubusercontent.com/saurograndi/healthcare_nlp/main/data/pubmed_diabetes_1000_meta.csv

In [3]:
import pandas as pd

df = pd.read_csv('pubmed_diabetes_1000_meta.csv')

In [4]:
df.head(5)

,seq,pubmed_id,abstract,pubdate,title,fulljournalname
0,0,PMC3312572,The pituitary gland and infundibulum can be in...,2012 Mar 18,Metastatic Prostate Adenocarcinoma Presenting ...,Case Reports in Medicine
1,1,PMC7730786,Current evidence suggests that the information...,2020 Dec 10,Different information needs in subgroups of pe...,BMC Public Health
2,2,PMC6913526,Ethnic minority populations exhibit disproport...,2019 Oct 16,"Development and Efficacy of an Electronic, Cul...",Journal of Medical Internet Research
3,3,PMC4653349,"Recently, imbalance in the vasopressin (AVP) s...",2016 Jan,Genetic vasopressin 1b receptor variance in ov...,European Journal of Endocrinology
4,4,PMC8606862,Deterioration in glucose homeostasis has been ...,2021 Nov 21,Progression of diabetes is associated with cha...,Physiological Reports


In [5]:
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import DocArrayInMemorySearch

loader = DataFrameLoader(df, page_content_column='abstract')
documents = loader.load()
documents[2]

Document(page_content='Ethnic minority populations exhibit disproportionately high rates of type 2 diabetes mellitus (T2DM). Electronic health tools have the potential to facilitate the cultural adaptation and tailoring of T2DM education to improve the knowledge and management of diabetes mellitus (DM).\nThis study aimed (1) to develop an adaptable Interactive Lifestyle Assessment, Counseling, and Education (I-ACE) software to support dietitian-delivered lifestyle counseling among low-socioeconomic status (SES) ethnic minority patients with T2DM and (2) to evaluate its effect on DM-related dietary knowledge and management compared with standard lifestyle advice (SLA) in a randomized controlled trial (RCT).\nThe I-ACE software, developed in consultation with clinical dieticians, incorporates evidence-based dietary and physical activity (PA) recommendations and educational materials. The features and behavioral change techniques include quantitative lifestyle (dietary intake and PA) asse

In [6]:
len(documents)

1000

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)

texts = r_splitter.split_documents(documents)

In [8]:
texts[1]

Document(page_content='lobe involvement may explain why patients with pituitary metastases frequently present with diabetes insipidus. We are presenting a case report of a 78-year-old male patient who had metastatic prostate with sudden onset of polyuria and persistent thirst. He had no electrolyte imbalance except mild hypernatremia. The MRI scan of the brain yielded a suspicious area in pituitary gland. A pituitary stalk metastasis was found on magnetic resonance imaging (MRI) of pituitary. Water deprivation test', metadata={'seq': 0, 'pubmed_id': 'PMC3312572', 'pubdate': '2012 Mar 18', 'title': 'Metastatic Prostate Adenocarcinoma Presenting Central Diabetes Insipidus', 'fulljournalname': 'Case Reports in Medicine'})

In [9]:
len(texts)

4498

### Embedding - Hugging Face

Hugging Face [Embedding leaderboard](https://huggingface.co/spaces/mteb/leaderboard)

In [10]:
!pip install -q InstructorEmbedding
!pip install -q sentence_transformers==2.2.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 981.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [11]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

hf_embeddings = HuggingFaceInstructEmbeddings(
    model_name = "hkunlp/instructor-base",
    model_kwargs = {'device': 'cpu'}
)

/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/115 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


### Vector Database - Chromadb

In [12]:
!pip install -q chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [13]:
from langchain.vectorstores import Chroma
persist_directory = 'db/chroma/'

!rm -rf ./db/chroma

In [14]:
%%time

chroma_vdb = Chroma.from_documents(
    documents=texts,
    embedding=hf_embeddings,
    persist_directory=persist_directory
)

CPU times: user 28min 55s, sys: 1min 26s, total: 30min 22s
Wall time: 30min 28s


In [15]:
# save to disk

chroma_vdb.persist()

In [ ]:
# Compress vector database directory
!zip -r chroma_vdb.zip ./db

In [14]:
# Delete vector database directory
!rm -rf db/

In [15]:
! wget -q https://raw.githubusercontent.com/saurograndi/healthcare_nlp/main/data/chroma_vdb.zip

In [16]:
!unzip chroma_vdb.zip

Archive:  chroma_vdb.zip
   creating: db/
   creating: db/chroma/
   creating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/
  inflating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/length.bin  
  inflating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/index_metadata.pickle  
  inflating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/header.bin  
  inflating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/link_lists.bin  
  inflating: db/chroma/264e3cef-3149-45b5-9f4d-30eb6cf32811/data_level0.bin  
  inflating: db/chroma/chroma.sqlite3  


In [17]:
%%time

chroma_vdb = Chroma(
    embedding_function=hf_embeddings,
    persist_directory=persist_directory
)

CPU times: user 1.3 s, sys: 49.4 ms, total: 1.35 s
Wall time: 1.47 s


In [18]:
query = "causes of diabetes"
docs = chroma_vdb.similarity_search(query, k=2)

In [19]:
docs

[Document(page_content='relevant pathway in human patients suffering from diabetes.', metadata={'fulljournalname': 'Molecular Metabolism', 'pubdate': '2018 Sep 19', 'pubmed_id': 'PMC6308908', 'seq': 761, 'title': 'Compensatory mechanisms for methylglyoxal detoxification in experimental & clinical diabetes'}),
 Document(page_content='diabetes-related comorbidities, and well-being.', metadata={'fulljournalname': 'BMC Public Health', 'pubdate': '2020 Dec 10', 'pubmed_id': 'PMC7730786', 'seq': 1, 'title': 'Different information needs in subgroups of people with diabetes mellitus: a latent class analysis'})]

In [20]:
retriever = chroma_vdb.as_retriever(search_kwargs={'k' : 10})

In [21]:
results = retriever.get_relevant_documents(query)

In [22]:
results

[Document(page_content='relevant pathway in human patients suffering from diabetes.', metadata={'fulljournalname': 'Molecular Metabolism', 'pubdate': '2018 Sep 19', 'pubmed_id': 'PMC6308908', 'seq': 761, 'title': 'Compensatory mechanisms for methylglyoxal detoxification in experimental & clinical diabetes'}),
 Document(page_content='diabetes-related comorbidities, and well-being.', metadata={'fulljournalname': 'BMC Public Health', 'pubdate': '2020 Dec 10', 'pubmed_id': 'PMC7730786', 'seq': 1, 'title': 'Different information needs in subgroups of people with diabetes mellitus: a latent class analysis'}),
 Document(page_content='Background: Diabetes is referred to a group of diseases characterized by high glucose levels in blood. It is caused by a deficiency in the production or function of insulin or both, which can occur because of different reasons, resulting in protein and lipid metabolic disorders. The aim of this study was to systematically review the prevalence and incidence of ty

In [23]:
pd.DataFrame([doc.metadata for doc in results])

,fulljournalname,pubdate,pubmed_id,seq,title
0,Molecular Metabolism,2018 Sep 19,PMC6308908,761,Compensatory mechanisms for methylglyoxal deto...
1,BMC Public Health,2020 Dec 10,PMC7730786,1,Different information needs in subgroups of pe...
2,Health Promotion Perspectives,2020 Mar 30,PMC7146037,7,Prevalence and incidence of type 1 diabetes in...
3,BMC Public Health,2019 Oct 15,PMC6794810,204,Association of genetic and behavioral characte...
4,Journal of Diabetes Investigation,2020 Jan 13,PMC7378450,787,Association between mean platelet volume in th...
5,Frontiers in Endocrinology,2021 Mar 11,PMC7992003,293,Aldose Reductase: An Emerging Target for Devel...
6,BMC Public Health,2020 Dec 10,PMC7730786,1,Different information needs in subgroups of pe...
7,BMJ Open Diabetes Research & Care,2022 Jan 25,PMC8796247,983,Need for improved diabetes support among peopl...
8,Human Mutation,2020 Feb 17,PMC7187370,616,Update of variants identified in the pancreati...
9,Human Mutation,2020 Feb 17,PMC7187370,616,Update of variants identified in the pancreati...


### LLM - OpenAI

In [24]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [25]:
!pip install -q --upgrade openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA # completion llm
llm = ChatOpenAI( openai_api_key=OPENAI_API_KEY, model_name='gpt-3.5-turbo-16k', temperature=0.0 )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [27]:
from langchain.prompts import PromptTemplate
new_template = """
Use the following pieces of context to answer the user's question. If you return an answer, end with 'It's my pleasure'. If you don't know the answer, just say that you don't know, don't try to make up an answer .
{context} Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(new_template)

In [28]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "The causes for diabetes?"
result = qa_chain({"query": question})

print(result['result'])

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


The causes for diabetes can vary and may include genetic factors, lifestyle characteristics, consumption of carbohydrate meals, medication side effects, and environmental factors. It is important to consider these factors in both future studies and diabetes management. It's my pleasure to assist you.


In [29]:
pd.DataFrame([doc.metadata for doc in result['source_documents']])

,fulljournalname,pubdate,pubmed_id,seq,title
0,Molecular Metabolism,2018 Sep 19,PMC6308908,761,Compensatory mechanisms for methylglyoxal deto...
1,Health Promotion Perspectives,2020 Mar 30,PMC7146037,7,Prevalence and incidence of type 1 diabetes in...
2,BMC Public Health,2020 Dec 10,PMC7730786,1,Different information needs in subgroups of pe...
3,BMC Public Health,2019 Oct 15,PMC6794810,204,Association of genetic and behavioral characte...
4,Frontiers in Endocrinology,2021 Mar 11,PMC7992003,293,Aldose Reductase: An Emerging Target for Devel...
5,Heliyon,2021 May 21,PMC8165422,839,Meta-analysis of studies on depression prevale...
6,BMC Public Health,2020 Dec 10,PMC7730786,1,Different information needs in subgroups of pe...
7,Journal of Diabetes Investigation,2020 Jan 13,PMC7378450,787,Association between mean platelet volume in th...
8,Bioscience Reports,2020 Sep 10,PMC7494983,252,Epigenetic modification and therapeutic target...
9,Human Mutation,2020 Feb 17,PMC7187370,616,Update of variants identified in the pancreati...


### LLM - Llama-2

In [30]:
!pip install -q ctransformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 50.6 MB/s eta 0:00:00


In [31]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
llm = CTransformers(
    model="TheBloke/Llama-2-7B-Chat-GGUF",
    model_file = 'llama-2-7b-chat.Q4_0.gguf',
    model_type="llama",
    config={'temperature': 0.01,
            #'max_new_tokens': 512,
            #'context_length': 500
            },
    callbacks=[StreamingStdOutCallbackHandler()]
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

llama-2-7b-chat.Q4_0.gguf:   0%|          | 0.00/3.83G [00:00<?, ?B/s]

In [32]:
# max context = 4096

retriever_k3 = chroma_vdb.as_retriever(search_type="mmr", search_kwargs={'k': 3, 'lambda_mult': 0.25})

In [33]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_k3,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "The causes for diabetes?"
result = qa_chain({"query": question})

print(result['result'])

 It's my pleasure to help you with that! Diabetes is a complex disease with multiple causes, including genetic mutations, obesity, lack of physical activity, and an unhealthy diet. In human patients suffering from diabetes, the following pathways have been implicated:
1. Insulin resistance: This occurs when the body's cells become less responsive to insulin, a hormone produced by the pancreas that regulates blood sugar levels.
2. Beta-cell dysfunction: The beta cells in the pancreas that produce insulin may not be able to produce enough insulin or may not produce it effectively, leading to high blood sugar levels.
3. Genetic mutations: Some people may inherit genetic mutations that affect their body's ability to regulate blood sugar levels. These mutations can affect the insulin receptor, the pancreatic beta cells, or other genes involved in glucose metabolism.
4. Obesity: Excess body weight is a major risk factor for developing type 2 diabetes, as it can

 lead

 to

 increase

 ins

...

ul

ates

 ins

ul

ates It's my pleasure to help you with that! Diabetes is a complex disease with multiple causes, including genetic mutations, obesity, lack of physical activity, and an unhealthy diet. In human patients suffering from diabetes, the following pathways have been implicated:
1. Insulin resistance: This occurs when the body's cells become less responsive to insulin, a hormone produced by the pancreas that regulates blood sugar levels.
2. Beta-cell dysfunction: The beta cells in the pancreas that produce insulin may not be able to produce enough insulin or may not produce it effectively, leading to high blood sugar levels.
3. Genetic mutations: Some people may inherit genetic mutations that affect their body's ability to regulate blood sugar levels. These mutations can affect the insulin receptor, the pancreatic beta cells, or other genes involved in glucose metabolism.
4. Obesity: Excess body weight is a major risk factor for developing type 2 diabetes, as it can lead to increase insulates

In [34]:
result['source_documents']

[Document(page_content='relevant pathway in human patients suffering from diabetes.', metadata={'fulljournalname': 'Molecular Metabolism', 'pubdate': '2018 Sep 19', 'pubmed_id': 'PMC6308908', 'seq': 761, 'title': 'Compensatory mechanisms for methylglyoxal detoxification in experimental & clinical diabetes'}),
 Document(page_content='The prevalence of diabetes mellitus is rising among children and adolescents worldwide. Cardiovascular diseases are the main cause of morbidity and mortality in diabetic patients. We review the impact of diabetes on establishing, during childhood and adolescence, the premises for cardiovascular diseases later in life. Interestingly, it seems that hyperglycemia is not the only factor that establishes an increased cardiovascular risk in adolescence. Other factors have been recognized to play a', metadata={'fulljournalname': 'International Journal of Molecular Sciences', 'pubdate': '2020 Jul 12', 'pubmed_id': 'PMC7403998', 'seq': 86, 'title': 'The Impact of Di

### LLM - Zephyr

In [37]:
from langchain.llms import CTransformers
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = CTransformers(
    model="TheBloke/zephyr-7B-beta-GGUF",
    model_file = 'zephyr-7b-beta.Q4_0.gguf',
    model_type="mistral",
    temperature = 0.01,
    #max_new_tokens = 250,
    #context_length = 500,
    # lib="avx2",
    callbacks=[StreamingStdOutCallbackHandler()]
)

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

zephyr-7b-beta.Q4_0.gguf:   0%|          | 0.00/4.11G [00:00<?, ?B/s]

In [38]:
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever_k3,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

question = "The causes for diabetes?"
result = qa_chain({"query": question})

print(result['result'])

 While hyperglycemia is a well-known risk factor for cardiovascular diseases in diabetic patients, recent studies have identified other factors that contribute to this increased risk during childhood and adolescence. ABCC8 and KCNJ11 are genes associated with both the protective against diabetes (GCK) and opposing phenotype (MAT2A). Further research is needed to fully understand the implications of these variants for individuals with pathogenic variants in these genes. It's my pleasure. While hyperglycemia is a well-known risk factor for cardiovascular diseases in diabetic patients, recent studies have identified other factors that contribute to this increased risk during childhood and adolescence. ABCC8 and KCNJ11 are genes associated with both the protective against diabetes (GCK) and opposing phenotype (MAT2A). Further research is needed to fully understand the implications of these variants for individuals with pathogenic variants in these genes. It's my pleasure.


In [39]:
result['source_documents']

[Document(page_content='relevant pathway in human patients suffering from diabetes.', metadata={'fulljournalname': 'Molecular Metabolism', 'pubdate': '2018 Sep 19', 'pubmed_id': 'PMC6308908', 'seq': 761, 'title': 'Compensatory mechanisms for methylglyoxal detoxification in experimental & clinical diabetes'}),
 Document(page_content='The prevalence of diabetes mellitus is rising among children and adolescents worldwide. Cardiovascular diseases are the main cause of morbidity and mortality in diabetic patients. We review the impact of diabetes on establishing, during childhood and adolescence, the premises for cardiovascular diseases later in life. Interestingly, it seems that hyperglycemia is not the only factor that establishes an increased cardiovascular risk in adolescence. Other factors have been recognized to play a', metadata={'fulljournalname': 'International Journal of Molecular Sciences', 'pubdate': '2020 Jul 12', 'pubmed_id': 'PMC7403998', 'seq': 86, 'title': 'The Impact of Di